In [1]:
import os
import numpy as np
import segyio

import segfast

In [2]:
INDEX_HEADERS_POSTSTACK = ('INLINE_3D', 'CROSSLINE_3D')
path = '/cubes/XXX_F3_NETHERLANDS/XXX_F3_NETHERLANDS.sgy'

file = segfast.open(path)

In [3]:
size = os.path.getsize(path) / (1024 ** 3)
print(f'Input SEG-Y file size: {size:3.2f} GB')

Input SEG-Y file size: 1.21 GB


In [4]:
headers = file.load_headers(INDEX_HEADERS_POSTSTACK)

In [5]:
for item in ['n_traces', 'sample_interval', 'delay']:
    print(item, getattr(file, item))

n_traces 619101
sample_interval 4000
delay 0


In [6]:
ilines, xlines = [np.unique(headers[index_header]) for index_header in INDEX_HEADERS_POSTSTACK]
ilines, xlines = [np.sort(item) for item in (ilines, xlines)]
index_values = headers[list(INDEX_HEADERS_POSTSTACK)].values
index_ordinals = index_values - index_values.min(axis=0).reshape(-1, 2)
idx_0, idx_1 = index_ordinals[:, 0], index_ordinals[:, 1]
lengths = (len(ilines), len(xlines))

mesh = []

for header in INDEX_HEADERS_POSTSTACK:
    values = headers[header]
    matrix = np.full(lengths, -1, dtype=values.dtype)
    matrix[idx_0, idx_1] = values
    mesh.append(matrix)

In [7]:
geometry = {
    'il': ilines,
    'xl': xlines,
    'nil': len(ilines),
    'nxl': len(xlines),
    'ntrace': file.n_traces,
    'mesh': mesh
}

In [8]:
file = segfast.open(path)

In [9]:
%%time

headers = file.load_headers(INDEX_HEADERS_POSTSTACK)

CPU times: user 17.7 ms, sys: 46.7 ms, total: 64.4 ms
Wall time: 93.2 ms


In [10]:
%%time

slide = file.load_depth_slices([300])

CPU times: user 1.05 s, sys: 137 ms, total: 1.19 s
Wall time: 1.08 s
